# Applications of AI in Medical Imaging
### Generation of synthetic data for medical imaging

In [46]:
import cv2

ModuleNotFoundError: No module named 'cv2'

In [47]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape, Conv2D, Conv2DTranspose, LeakyReLU, Dropout
import numpy as np
import matplotlib.pyplot as plt
import os

In [48]:


image_directory_stroke = "/home/m2225046/Desktop/hw5/archive/Brain_Data_Organised/Stroke/"

image_directory_normal = "/home/m2225046/Desktop/hw5/archive/Brain_Data_Organised/Normal/"

In [ ]:


def rename_images(folder_path, prefix="img"):
    """
    Renames all .jpg files in the specified folder to a unique identifier format.
    E.g., img001.jpg, img002.jpg, ...

    Args:
        folder_path (str): Path to the folder containing .jpg images.
        prefix (str): Prefix for the renamed files (default is "img").
    """
    # Get all files in the folder
    files = [f for f in os.listdir(folder_path) if f.endswith(".jpg")]
    
    # Sort the files to ensure consistent ordering (optional)
    files.sort()

    # Rename each file
    for idx, file in enumerate(files, start=1):
        # Create a new filename with a unique identifier
        new_name = f"{prefix}{str(idx).zfill(3)}.jpg"  # e.g., img.001.jpg
        old_path = os.path.join(folder_path, file)
        new_path = os.path.join(folder_path, new_name)

        # Rename the file
        os.rename(old_path, new_path)
        print(f"Renamed: {file} -> {new_name}")

# Specify the folder containing the images
folder_path = "/home/m2225046/Desktop/hw5/archive/Brain_Data_Organised/Stroke/"
folder_path_2 = "/home/m2225046/Desktop/hw5/archive/Brain_Data_Organised/Normal/"
folder_path_3 = "/home/m2225046/Desktop/hw5/archive/Brain_Data_Organised/Test/"
# Call the function
rename_images(folder_path_2, prefix="normal_img")

Renamed: normal_img001.jpg -> normal_img001.jpg
Renamed: normal_img002.jpg -> normal_img002.jpg
Renamed: normal_img003.jpg -> normal_img003.jpg
Renamed: normal_img004.jpg -> normal_img004.jpg
Renamed: normal_img005.jpg -> normal_img005.jpg
Renamed: normal_img006.jpg -> normal_img006.jpg
Renamed: normal_img007.jpg -> normal_img007.jpg
Renamed: normal_img008.jpg -> normal_img008.jpg
Renamed: normal_img009.jpg -> normal_img009.jpg
Renamed: normal_img010.jpg -> normal_img010.jpg
Renamed: normal_img011.jpg -> normal_img011.jpg
Renamed: normal_img012.jpg -> normal_img012.jpg
Renamed: normal_img013.jpg -> normal_img013.jpg
Renamed: normal_img014.jpg -> normal_img014.jpg
Renamed: normal_img015.jpg -> normal_img015.jpg
Renamed: normal_img016.jpg -> normal_img016.jpg
Renamed: normal_img017.jpg -> normal_img017.jpg
Renamed: normal_img018.jpg -> normal_img018.jpg
Renamed: normal_img019.jpg -> normal_img019.jpg
Renamed: normal_img020.jpg -> normal_img020.jpg
Renamed: normal_img021.jpg -> normal_img

In [49]:
## Step 1: Load and Preprocess Data

data_path = "/home/m2225046/Desktop/hw5/archive/Brain_Data_Organised/Stroke/"

def load_images(data_path, img_size=(64, 64)):
    images = []
    for filename in os.listdir(data_path):
        if filename.endswith(".jpg"):
            img = tf.keras.preprocessing.image.load_img(os.path.join(data_path, filename), target_size=img_size)
            img = tf.keras.preprocessing.image.img_to_array(img)
            images.append(img)
    images = np.array(images)
    images = (images - 127.5) / 127.5  # Normalize images to [-1, 1]
    return images

images = load_images(data_path)

In [50]:
# Step 2: Define GAN Components
# Generator
def build_generator():
    model = Sequential([
        Dense(8 * 8 * 256, input_dim=100),  # Input: Random noise (latent space)
        LeakyReLU(0.2),
        Reshape((8, 8, 256)),
        Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        LeakyReLU(0.2),
        Conv2DTranspose(64, (4, 4), strides=(2, 2), padding="same"),
        LeakyReLU(0.2),
        Conv2DTranspose(3, (4, 4), strides=(2, 2), padding="same", activation="tanh")
    ])
    return model

In [51]:
# Discriminator
def build_discriminator():
    model = Sequential([
        Conv2D(64, (4, 4), strides=(2, 2), padding="same", input_shape=(64, 64, 3)),
        LeakyReLU(0.2),
        Dropout(0.3),
        Conv2D(128, (4, 4), strides=(2, 2), padding="same"),
        LeakyReLU(0.2),
        Dropout(0.3),
        Flatten(),
        Dense(1, activation="sigmoid")
    ])
    return model


In [52]:
# Step 3: Compile GAN
generator = build_generator()
discriminator = build_discriminator()
discriminator.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Combine Generator and Discriminator
discriminator.trainable = False
gan = Sequential([generator, discriminator])
gan.compile(optimizer="adam", loss="binary_crossentropy")

In [53]:
# Step 4: Train the GAN
def train_gan(images, epochs=10000, batch_size=64, save_interval=1000):
    half_batch = batch_size // 2

    for epoch in range(epochs):
        # Train Discriminator
        idx = np.random.randint(0, images.shape[0], half_batch)
        real_imgs = images[idx]

        noise = np.random.normal(0, 1, (half_batch, 100))
        fake_imgs = generator.predict(noise)

        real_labels = np.ones((half_batch, 1))
        fake_labels = np.zeros((half_batch, 1))

        d_loss_real = discriminator.train_on_batch(real_imgs, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_imgs, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train Generator
        noise = np.random.normal(0, 1, (batch_size, 100))
        valid_labels = np.ones((batch_size, 1))
        g_loss = gan.train_on_batch(noise, valid_labels)

        # Print progress
        if epoch % 100 == 0:
            print(f"Epoch {epoch} | D Loss: {d_loss[0]} | G Loss: {g_loss}")

        # Save generated images at intervals
        if epoch % save_interval == 0:
            save_generated_images(epoch, generator)

In [54]:
def save_generated_images(epoch, generator, output_dir="gan_images", examples=10, dim=(1, 10), figsize=(20, 2)):
    os.makedirs(output_dir, exist_ok=True)
    noise = np.random.normal(0, 1, (examples, 100))
    generated_images = generator.predict(noise)
    generated_images = (generated_images + 1) / 2.0  # Rescale to [0, 1]

    fig, axs = plt.subplots(*dim, figsize=figsize)
    for i in range(examples):
        axs[i].imshow(generated_images[i])
        axs[i].axis("off")
    fig.savefig(f"{output_dir}/generated_{epoch}.png")
    plt.close()


In [ ]:
# Step 5: Start Training
train_gan(images, epochs=5000, batch_size=32, save_interval=500)


In [37]:
# Step 2: Build a Simple CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # For binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 3: Train the Model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Step 4: Evaluate the Model
val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")

ValueError: Asked to retrieve element 0, but the Sequence has length 0

In [36]:
#Train Model
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=10
)


ValueError: Unexpected value for `steps_per_epoch`. Received value is 0. Please check the docstring for `model.fit()` for supported values.